# 2D Uplift model

This model uses a stress boundary condition to force an uplift.

This model also utilises scaling our numbers into dimensionless units.

<table><tr><td><img src='../images/uplift_init.png'></td><td><img src='../images/uplift.png'></td></tr></table>


In [ ]:
import numpy as np
import underworld as uw
import math
from underworld import function as fn
import glucifer
import os

from unsupported.scaling import *
from unsupported.scaling import nonDimensionalize as nd

outputPath='output/'

# create outputPath
import glob, json, os
# make a unique path
if os.path.exists(outputPath):
    outputPath += '_'+str(len(glob.glob(outputPath+str('*')))-1)

# build output dir string
if not outputPath.endswith('/'):
    outputPath += '/'
    
# make the output path
if not os.path.exists(outputPath):
    os.makedirs(outputPath)

In [ ]:
# reference units
KL_meters = 100e3 * u.meter
K_viscosity = (1e16 * u.pascal * u.second).to_base_units()
K_density = (3.3e3 * u.kilogram / (u.meter)**3 ).to_base_units()

In [ ]:
KM_kilograms = K_density * KL_meters**3
KT_seconds = KM_kilograms / ( KL_meters * K_viscosity )
K_substance = 1. * u.mole
Kt_degrees = 1. * u.kelvin

scaling = {"[time]": KT_seconds,
           "[length]": KL_meters, 
           "[mass]": KM_kilograms,
           "[temperature]": Kt_degrees,
           "[substance]": K_substance}

In [ ]:
# all nondimensional units
gravity = nd(9.81 * u.meter / u.second**2, scaling)
density = nd( 3300 * u.kilogram / u.meter**3, scaling)
viscosity = nd( 1e22 * u.Pa * u.sec, scaling)
bulk_visc = nd( 1e11 * u.Pa *u.sec, scaling)

In [ ]:
Lx = nd( 100e3 * u.meter, scaling)
Ly = nd(  60e3 * u.meter, scaling)
dx = nd(   5e3 * u.meter, scaling)
dy = nd(   5e3 * u.meter, scaling)
center = nd(50e3 * u.meter, scaling)
width = nd(3e3*u.meter, scaling)

In [ ]:
lithostaticPressure = 0.6*Ly*density*gravity

In [ ]:
resUnit = 5
boxLength = Lx
boxHeight = Ly
elType = "Q1/dQ0"
resx = 100
resy = 60
minCoord = [0.,0.]
maxCoord = [boxLength,boxHeight]

In [ ]:
mesh = uw.mesh.FeMesh_Cartesian( elementType = (elType), 
                                 elementRes  = (resx, resy), 
                                 minCoord    = minCoord, 
                                 maxCoord    = maxCoord )

velocityField = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=mesh.dim )
stressField   = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=3 )
pressureField = uw.mesh.MeshVariable( mesh=mesh.subMesh, nodeDofCount=1 )

In [ ]:
velocityField.data[:] = [0.,0.]
pressureField.data[:] = 0.
stressField.data[:] = [0.0,0.0,0.0]

for ii in mesh.specialSets['MinJ_VertexSet']:
    coord = mesh.data[ii]
    stressField.data[ii] = [0.0,-lithostaticPressure*(1.+0.2*np.exp((-1/width*(coord[0]-center)**2))), 0.0]
#     stressField.data[ii] = [0.0,-lithostaticPressure*(1.+0.2*np.sin(coord[0]*(2.*np.pi)) ), 0.0]

In [ ]:
# visualise the bottom stress condition
if uw.rank() == 0:
    uw.matplotlib_inline()
    import matplotlib.pyplot as pyplot
    import matplotlib.pylab as pylab
    pyplot.ion()
    pylab.rcParams[ 'figure.figsize'] = 12, 6
    xcoord = Dimensionalize(mesh.data[mesh.specialSets['MinJ_VertexSet'].data][:,0],scaling, u.kilometer)
    stress = Dimensionalize(stressField.data[mesh.specialSets['MinJ_VertexSet'].data][:,1], scaling, u.gigaPa)
    pyplot.plot( xcoord, stress, 'o', color = 'black', label='numerical') 
    pyplot.xlabel('Y coords: km')
    pyplot.ylabel('Pressure: GPa')
    pyplot.show()

In [ ]:
# Initialise a swarm.
swarm = uw.swarm.Swarm( mesh=mesh, particleEscape=True )
advector= uw.systems.SwarmAdvector(velocityField, swarm, order=2)

# Add a data variable which will store an index to determine material.
materialVariable = swarm.add_variable( dataType="double", count=1 )

# Create a layout object that will populate the swarm across the whole domain.
swarmLayout = uw.swarm.layouts.PerCellSpaceFillerLayout( swarm=swarm, particlesPerCell=20 )
# Populate.
swarm.populate_using_layout( layout=swarmLayout )

materialVariable.data[:]=0
for index,coord in enumerate(swarm.particleCoordinates.data):
    if coord[1] < boxHeight*0.6:
        materialVariable.data[index]=1

# population control regulars particle creation and deletion
# important for inflow/outflow problems
population_control = uw.swarm.PopulationControl(swarm, 
                                                aggressive=True,splitThreshold=0.15, maxDeletions=2,maxSplits=10,
                                                particlesPerCell=20)

# build tracer swarm for fluid level
mswarm = uw.swarm.Swarm( mesh=mesh, particleEscape=True )
# msAdvector= uw.systems.SwarmAdvector(velocityField, mswarm, order=2)

# initial height at 'air' level
particleCoordinates = np.zeros((100,2))
particleCoordinates[:,0] = np.linspace(0.0, Lx, len(particleCoordinates))#0.5*Lx
particleCoordinates[:,1] = 0.6*Ly
ignore=mswarm.add_particles_with_coordinates(particleCoordinates)

In [ ]:
# create a scaling factor
cm_per_year = Dimensionalize(1,scaling,u.centimeter/u.year)

fig1 = glucifer.Figure(rulers=True, boundingBox=((0.0, 0.0), (Lx, Ly)))
fig1.append( glucifer.objects.Points(swarm, materialVariable, fn_size=2., colourBar=False ) )
fig1.append( glucifer.objects.Points(mswarm, fn.misc.constant(0.0), fn_size=4., colours="black",colourBar=False) )
fig1.append( glucifer.objects.VectorArrows(mesh, cm_per_year.magnitude*0.1*velocityField) )

fig1.show()

In [ ]:
# Lambda = materialVariable * fn.misc.constant(10.0) -  is created for pseudo compressibility
lambdaFn = uw.function.branching.map( fn_key=materialVariable, 
                                    mapping={ 0: bulk_visc, 1: 0.0 } )

densityFn = uw.function.branching.map( fn_key=materialVariable, 
                                    mapping={ 0: 0.0, 1: density } )

forceFn = densityFn * (0.0,-gravity)

In [ ]:
iWalls = mesh.specialSets["MinI_VertexSet"] + mesh.specialSets["MaxI_VertexSet"]
jWalls = mesh.specialSets["MinJ_VertexSet"] + mesh.specialSets["MaxJ_VertexSet"]
bottomWall = mesh.specialSets["MinJ_VertexSet"]
allWalls = iWalls + jWalls

# Now, using these sets, decide which degrees of freedom (on each node) should be considered Dirichlet.
stokesBC = uw.conditions.DirichletCondition( variable      = velocityField, 
                                             indexSetsPerDof = (iWalls, jWalls-bottomWall) )

# add neumann bcs
nbc      = uw.conditions.NeumannCondition( flux=stressField, variable = velocityField, 
                                             nodeIndexSet = bottomWall )

**Create a Stokes system**

In [ ]:
stokesPIC = uw.systems.Stokes( velocityField = velocityField, 
                               pressureField = pressureField,
                               conditions    = [stokesBC, nbc],
                               fn_viscosity  = viscosity, 
                               fn_bodyforce  = forceFn,
                               fn_lambda     = lambdaFn )
solver = uw.systems.Solver( stokesPIC )
solver.set_inner_method("lu")

In [ ]:
vdotv = fn.math.dot(velocityField,velocityField)
v2sum_integral  = uw.utils.Integral( mesh=mesh, fn=vdotv )
volume_integral = uw.utils.Integral( mesh=mesh, fn=1. )
velmag = fn.math.sqrt(vdotv)

tracer_height = fn.view.min_max(fn.coord()[1]) # parallel safe

In [ ]:
steps         = 0
time          = 0
finalTime     = nd(2 * u.megayear, scaling)

fieldDict = {'velocity':velocityField, 'pressure':pressureField}
swarmDict = {'material':materialVariable}

In [ ]:
outfile = open(outputPath+'buildMount.txt', 'w+')
string = "steps, timestep, vrms, change in peak height"
print(string)
outfile.write( string+"\n")

# initialise loop
dt = -1

tracer_height.evaluate(mswarm)
h1 = tracer_height.max_global()

while time<finalTime:
    # Get solution
    solver.solve()
    
    # calculate metrics
    v2int = v2sum_integral.evaluate()[0]
    vol   = volume_integral.evaluate()[0]
    
    # get time step first time around
    if dt < 0:
        dt = advector.get_max_dt()
    h0 = h1
    
    fig1.save(outputPath+"particals-"+str(steps)+".png")

    # update peak heigh
    tracer_height.evaluate(mswarm)
    h1 = tracer_height.max_global()
    
    diffH = h1-h0
    string = "{}, {}, {}, {}".format(steps,
                                     Dimensionalize(dt, scaling, u.year),
                                     Dimensionalize(np.sqrt(v2int/vol), scaling, u.cm/u.year),
                                     Dimensionalize(diffH,scaling, u.metre) )
    print(string)
    outfile.write(string+"\n")
    
    # Advect particles   
    advector.integrate(dt)  
#     msAdvector.integrate(dt)

    # by hand advection. Not parallel safe ... fix pending
    with mswarm.deform_swarm():
        mswarm.particleCoordinates.data[:,1] += dt*velocityField.evaluate(mswarm)[:,1]

        
    # population control
    population_control.repopulate()
    
    steps += 1
    time += dt


outfile.close()

In [ ]:
fig2 = glucifer.Figure()
fig2.append( glucifer.objects.Surface(mesh, cm_per_year.magnitude*velmag) )

fig2.show()

In [ ]:
fig1.show()

In [ ]:
# visualise the bottom stress condition
if uw.rank() == 0:
    uw.matplotlib_inline()
    import matplotlib.pyplot as pyplot
    import matplotlib.pylab as pylab
    pyplot.ion()
    pylab.rcParams[ 'figure.figsize'] = 12, 6
    xcoord = Dimensionalize(mswarm.particleCoordinates.data[:,0], scaling, u.kilometer)
    ycoord = Dimensionalize(mswarm.particleCoordinates.data[:,1], scaling, u.kilometer)
    pyplot.plot( xcoord, ycoord, 'o', color = 'black', label='numerical') 
    pyplot.xlabel('X coord: km')
    pyplot.ylabel('Y coord: km')
    pyplot.show()

**Exercise:** 
1. Change the stress boundary condition and watch the topography change